In [1]:
%matplotlib inline
from scipy.stats import kurtosis, skew
import datetime
import numpy as np
import pandas as pd
from pandas_datareader import data
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
# Import 11 SPDRs price
spdrs_list = ['XLB','XLC','XLE','XLF','XLI','XLK','XLP','XLRE','XLU','XLV','XLY']     # XLC ==> VOX, XLRE ==> IYR
# start and end date
spdrs_stat_date = datetime.datetime(2014, 1, 1)
spdrs_end_date = datetime.datetime(2019, 12, 31)
# create empty dataframe
spdrs_price_df = pd.DataFrame()

# Select only adj close data for all SPDR
for spdrs_name in spdrs_list:        
    df = data.get_data_yahoo(spdrs_name, spdrs_stat_date, spdrs_end_date)
    spdrs_price_df[spdrs_name] = df['Adj Close']
    
spdrs_price_df.head(3)

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2013-12-31,39.781387,NaN,66.326424,15.372337,45.100849,31.869316,35.226295,NaN,29.487286,49.048355,60.550556
2014-01-02,39.462929,NaN,65.382210,15.287949,44.470848,31.557217,34.791908,NaN,29.036858,48.774105,60.269672
2014-01-03,39.368259,NaN,65.142441,15.393431,44.591675,31.405630,34.726334,NaN,28.943678,48.889114,60.106598


In [3]:
# Check data ==> all is non-null
spdrs_price_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1511 entries, 2013-12-31 to 2019-12-31
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   XLB     1511 non-null   float64
 1   XLC     387 non-null    float64
 2   XLE     1511 non-null   float64
 3   XLF     1511 non-null   float64
 4   XLI     1511 non-null   float64
 5   XLK     1511 non-null   float64
 6   XLP     1511 non-null   float64
 7   XLRE    1065 non-null   float64
 8   XLU     1511 non-null   float64
 9   XLV     1511 non-null   float64
 10  XLY     1511 non-null   float64
dtypes: float64(11)
memory usage: 141.7 KB


### We'll find that XLRE and XLC don't have enough information between 1st Jan 2014 and 31st Dec 2019
### So we need to find new fund to replace  XLRE and XLC.

## Find new fund to replace  XLRE (real estate sector)

Definitive List Of Real Estate ETFs : https://etfdb.com/etfdb-category/real-estate/

In [4]:
# Import Definitive List Of Real Estate ETFs as mentioned above

# List Of Real Estate ETFs as mentioned above
real_estate_list = ['VNQ','SCHH','IYR','XLRE','ICF', 'USRT','RWR','FREL','SRVR','REM','BBRE','REZ','SRET','KBWY', 
                    'MORT','INDS','FRI','PPTY','VRAI','PSR','NETL','EWRE','NURE','ROOF','RDOG', 'REK']

# start and end date for data import
real_estate_stat_date = datetime.datetime(2015, 10, 9) 
real_estate_end_date = datetime.datetime(2019, 12, 31)

# Create empty dataframe
real_estate_return_df = pd.DataFrame()

# Find  log return of all Real Estate ETFs
for real_estate_name in real_estate_list: 
    # import data 
    df2 = data.get_data_yahoo(real_estate_name, real_estate_stat_date, real_estate_end_date)
    # compute log return
    df2['log_return'] = np.log(df2['Adj Close'].pct_change() + 1 )
    # delte NaN
    df2 = df2.dropna()
    # Create column name of real_estate_return_df
    real_estate_return_df[real_estate_name + '_log_return'] = df2['log_return']
    
real_estate_return_df.head()

,VNQ_log_return,SCHH_log_return,IYR_log_return,XLRE_log_return,ICF_log_return,USRT_log_return,RWR_log_return,FREL_log_return,SRVR_log_return,REM_log_return,...,FRI_log_return,PPTY_log_return,VRAI_log_return,PSR_log_return,NETL_log_return,EWRE_log_return,NURE_log_return,ROOF_log_return,RDOG_log_return,REK_log_return
Date,,,,,,,,,,,,,,,,,,,,,
2015-10-09,-0.001645,-0.001537,-0.001077,-0.001657,-0.001863,0.001086,-0.001992,-0.000447,NaN,-0.001925,...,-0.001371,NaN,NaN,0.001250,NaN,0.000000,NaN,0.000403,-0.005357,0.001503
2015-10-12,0.007064,0.006647,0.005373,0.006280,0.005684,0.005845,0.006297,0.004462,NaN,0.002886,...,0.005474,NaN,NaN,0.006502,NaN,0.000000,NaN,0.004827,0.005821,-0.005020
2015-10-13,-0.007824,-0.008443,-0.007395,-0.006280,-0.006825,-0.007366,-0.007406,-0.007598,NaN,-0.008683,...,-0.007306,NaN,NaN,-0.006780,NaN,0.005560,NaN,-0.011705,-0.009799,0.008519
2015-10-14,-0.007503,-0.008515,-0.006499,0.000000,-0.008754,-0.003267,-0.008355,-0.006301,NaN,-0.006806,...,-0.009671,NaN,NaN,-0.007106,NaN,-0.013156,NaN,-0.005292,0.000234,0.006466
2015-10-15,0.013566,0.014152,0.011345,-0.002988,0.014341,0.006740,0.014328,0.009928,NaN,0.003895,...,0.015155,NaN,NaN,0.006411,NaN,0.000000,NaN,0.012574,0.012347,-0.012974


In [5]:
# Find correlation with XLRE and Sort in descending order and Disregard XLRE
real_estate_corr_matrix = real_estate_return_df.corr()
# Drop/Delete XLRE
real_estate_max2min_corr = real_estate_corr_matrix['XLRE_log_return'].drop('XLRE_log_return').sort_values(ascending=False)
real_estate_max2min_corr

FREL_log_return    0.938755
IYR_log_return     0.938694
ICF_log_return     0.936888
PPTY_log_return    0.933409
VNQ_log_return     0.932438
USRT_log_return    0.931376
SCHH_log_return    0.921609
FRI_log_return     0.920977
RWR_log_return     0.920557
REZ_log_return     0.883942
NETL_log_return    0.875707
PSR_log_return     0.843562
SRVR_log_return    0.820925
ROOF_log_return    0.814926
BBRE_log_return    0.793688
RDOG_log_return    0.768477
INDS_log_return    0.755084
KBWY_log_return    0.737135
EWRE_log_return    0.679781
SRET_log_return    0.679534
NURE_log_return    0.636016
REM_log_return     0.485818
MORT_log_return    0.472028
VRAI_log_return    0.388176
REK_log_return    -0.923344
Name: XLRE_log_return, dtype: float64

In [6]:
# Find ETF that has strongest correlation with XLRE and has completely historical price data
real_estate_list = ['VNQ','SCHH','IYR','XLRE','ICF', 'USRT','RWR','FREL','SRVR','REM','BBRE','REZ','SRET','KBWY', 
                    'MORT','INDS','FRI','PPTY','VRAI','PSR','NETL','EWRE','NURE','ROOF','RDOG', 'REK']

# start and end date
real_estate_stat_date = datetime.datetime(2014, 1, 1)
real_estate_end_date = datetime.datetime(2019, 12, 31)

# Create empty data frame
real_estate_price_df = pd.DataFrame()

# import all real estate ETFs and select only adj close data
for real_estate_name in real_estate_list: 
    # import all real estate ETFs and select only adj close data
    df = data.get_data_yahoo(real_estate_name, real_estate_stat_date, real_estate_end_date)
    # select only adj close data
    real_estate_price_df[real_estate_name] = df['Adj Close']

# create empty list for keep
real_estate_candidate = []

for i in real_estate_max2min_corr.index:
    # cut '_log_return' and keep only ETF's name (IYR_log_return ==> IYR)
    cut_string = i.split('_')[0]
    # check that data have NaN or not, then keep ETF to real_estate_candidate
    if real_estate_price_df[cut_string].isnull().sum() == 0:
        real_estate_candidate.append(cut_string)
        
# Show result, select ETF that has highest correlation with XLRE       
print(f'We will replace XLRE with {real_estate_candidate[0]}')
print('because it has strongest correlation with XLRE and historical data is complete for all period.')

We will replace XLRE with IYR
because it has strongest correlation with XLRE and historical data is complete for all period.


## Find new fund to replace  XLC (communication services sector)

Alternative ETFs in the FactSet Equity: U.S. - Large Cap Growth Segment / Alternative ETFs in the ETF Database Large Cap Growth Equities Category
https://etfdb.com/etf/XLC/#etf-ticker-profile

Definitive List Of Communications Equities ETFs : https://etfdb.com/etfdb-category/communications-equities/

Telecom ETF List : https://etfdb.com/etfs/sector/telecom/

In [7]:
# Import communication services and telecommunication sector as mentioned above and in the url
communication_list = ['XLC','VOX','NXTG','FCOM','IYZ','IXP','XTL','WBIF','EWCO','LTL','IGN','PSCU','SPY','SFY']

# start and end date for import data
communication_stat_date = datetime.datetime(2018, 6, 19)
communication_end_date = datetime.datetime(2019, 12, 31)

# Create empty dateaframe
communication_return_df = pd.DataFrame()

# calculate log return for each etf
for communication_name in communication_list:  
    # import all communication services and telecommunication sector fund
    df4 = data.get_data_yahoo(communication_name, communication_stat_date, communication_end_date)
    # calculate log return
    df4['log_return'] = np.log(df4['Adj Close'].pct_change() + 1 )
    df4 = df4.dropna()
    communication_return_df[communication_name + '_log_return'] = df4['log_return']
    
communication_return_df.head()

,XLC_log_return,VOX_log_return,NXTG_log_return,FCOM_log_return,IYZ_log_return,IXP_log_return,XTL_log_return,WBIF_log_return,EWCO_log_return,LTL_log_return,IGN_log_return,PSCU_log_return,SPY_log_return,SFY_log_return
Date,,,,,,,,,,,,,,
2018-06-20,0.012334,-0.000575,0.007609,-0.010424,-0.001436,-0.004448,0.000271,-0.000416,NaN,-0.001872,0.008614,0.005882,0.001705,NaN
2018-06-21,-0.006148,-0.008316,-0.006367,-0.010534,-0.008296,0.000000,-0.015555,-0.000486,NaN,-0.021879,-0.012383,-0.010317,-0.006289,NaN
2018-06-22,0.004367,0.006128,-0.003836,0.008086,0.001809,0.011082,-0.004134,0.000694,NaN,0.000000,-0.008532,0.002959,0.001822,NaN
2018-06-25,-0.020813,-0.012645,-0.017960,-0.008086,-0.011271,-0.008300,-0.014184,-0.016978,NaN,0.000000,-0.017091,0.003502,-0.013707,NaN
2018-06-26,0.001657,0.001283,-0.000618,0.002468,0.003932,-0.002040,0.012526,0.009761,NaN,0.000000,0.007967,0.001838,0.002212,NaN


In [8]:
# Find correlation with XLC and Sort in descending order and Disregard XLC
communication_corr_matrix = communication_return_df.corr()
# Drop/Delete XLC
communication_max2min_corr = communication_corr_matrix['XLC_log_return'].drop('XLC_log_return').sort_values(ascending=False)
communication_max2min_corr

VOX_log_return     0.973083
IXP_log_return     0.891115
EWCO_log_return    0.870260
SFY_log_return     0.862464
SPY_log_return     0.853976
FCOM_log_return    0.805299
XTL_log_return     0.742080
IYZ_log_return     0.732447
IGN_log_return     0.727194
NXTG_log_return    0.612008
PSCU_log_return    0.519668
WBIF_log_return    0.492289
LTL_log_return     0.460469
Name: XLC_log_return, dtype: float64

In [12]:
# Find ETF that has strongest correlation with XLRE and has completely historical price data
communication_list = ['XLC','VOX','NXTG','FCOM','IYZ','IXP','XTL','WBIF','EWCO','LTL','IGN','PSCU','SPY','SFY']

# Start and End date
communication_stat_date = datetime.datetime(2014, 1, 1) 
communication_end_date = datetime.datetime(2019, 12, 31)

# Create empty dataframe
communication_price_df = pd.DataFrame()

for communication_name in communication_list:   
    # import funds in communication sector
    df3 = data.get_data_yahoo(communication_name, communication_stat_date, communication_end_date)
    # select only adj close data
    communication_price_df[communication_name] = df3['Adj Close']

# Create empty list
communication_candidate = []

for i in communication_max2min_corr.index:
    # cut '_log_return' and keep only ETF's name (IYR_log_return ==> IYR)
    cut_string = i.split('_')[0]
    # check that data have NaN or not, then keep ETF to communication_candidate
    if communication_price_df[cut_string].isnull().sum() == 0:
        communication_candidate.append(cut_string)
        
# Show result, select ETF that has highest correlation with XLC         
print(f'We will replace XLC with {communication_candidate[0]}')
print('because it has strongest correlation with XLC and historical data is complete for all period.')

We will replace XLC with VOX
because it has strongest correlation with XLC and historical data is complete for all period.


## Now, we will get 11 SPDRs ETF for our process
## (replace XLC with VOX and replace XLRE with IYR)

In [13]:
# import 11 spdrs ETF again but this time we replace XLRE with IYR and replace XLC with VOX
spdrs_list = ['XLB','VOX','XLE','XLF','XLI','XLK','XLP','IYR','XLU','XLV','XLY']

# start and end date
spdrs_stat_date = datetime.datetime(2014, 1, 1)
spdrs_end_date = datetime.datetime(2019, 12, 31)

# Create empty dateframe
spdrs_price_df = pd.DataFrame()

# Import data and select only adj close data
for spdrs_name in spdrs_list:  
    # import
    df = data.get_data_yahoo(spdrs_name, spdrs_stat_date, spdrs_end_date)
    # select only adj data
    spdrs_price_df[spdrs_name] = df['Adj Close']
    
spdrs_price_df

,XLB,VOX,XLE,XLF,XLI,XLK,XLP,IYR,XLU,XLV,XLY
Date,,,,,,,,,,,
2013-12-31,39.781387,72.149460,66.326424,15.372337,45.100849,31.869316,35.226295,48.451271,29.487286,49.048355,60.550556
2014-01-02,39.462929,71.528519,65.382210,15.287949,44.470848,31.557217,34.791908,48.374454,29.036858,48.774105,60.269672
2014-01-03,39.368259,71.321556,65.142441,15.393431,44.591675,31.405630,34.726334,48.658653,28.943678,48.889114,60.106598
2014-01-06,39.153088,71.571640,65.232361,15.407503,44.332764,31.352112,34.595215,48.858345,28.990271,48.685623,59.735107
2014-01-07,39.084229,72.080452,65.726936,15.414529,44.608921,31.637468,34.783710,49.027328,29.246548,49.198750,60.097538
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,59.291908,92.998398,55.247555,29.654032,79.649132,89.694038,60.466576,88.187439,60.928539,99.836365,122.781921
2019-12-26,59.553837,93.719543,55.229717,29.818348,79.834190,90.362526,60.524250,88.658531,61.033138,99.768501,124.280457
2019-12-27,59.330715,93.541725,54.997887,29.741024,79.765999,90.352684,60.793369,88.898865,61.213787,99.797585,124.270607


In [14]:
# Check that all data is non-null
spdrs_price_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1511 entries, 2013-12-31 to 2019-12-31
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   XLB     1511 non-null   float64
 1   VOX     1511 non-null   float64
 2   XLE     1511 non-null   float64
 3   XLF     1511 non-null   float64
 4   XLI     1511 non-null   float64
 5   XLK     1511 non-null   float64
 6   XLP     1511 non-null   float64
 7   IYR     1511 non-null   float64
 8   XLU     1511 non-null   float64
 9   XLV     1511 non-null   float64
 10  XLY     1511 non-null   float64
dtypes: float64(11)
memory usage: 141.7 KB
